# Descargar Dataset

Dataset con el que vamos a trabajar:
https://www.kaggle.com/datasets/javagarm/fifa-19-complete-player-dataset

Inicialmente se debe tener la key de la API de Kaggle para descargar localmente el dataset. Para esto se puede seguir esta guía :

https://saturncloud.io/blog/how-to-use-kaggle-datasets-in-google-colab/

**Nota:** Solo es necesario seguirla hasta el paso 2, una vez descargado el Json con la Key se pueden empezar a ejecutar las celdas

In [ ]:
# Run this cell and select the kaggle.json file downloaded
# from the Kaggle account settings page.
from google.colab import files
files.upload()

Saving kaggle.json to kaggle (1).json


{'kaggle (1).json': b'{"username":"sebassaldarriaga","key":"3bb04850fe5fccc1075f237932526fb6"}'}

In [ ]:
# Let's make sure the kaggle.json file is present.
!ls -lha kaggle.json

-rw-r--r-- 1 root root 72 Oct 21 23:20 kaggle.json


In [ ]:
# Next, install the Kaggle API client.
!pip install -q kaggle

In [ ]:
# The Kaggle API client expects this file to be in ~/.kaggle,
# so move it there.
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

# This permissions change avoids a warning on Kaggle tool startup.
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d javagarm/fifa-19-complete-player-dataset

fifa-19-complete-player-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
!unzip fifa-19-complete-player-dataset.zip

Archive:  fifa-19-complete-player-dataset.zip
replace kl.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: kl.csv                  


In [ ]:
import pandas as pd
import sklearn as skl
import numpy as np

In [ ]:
df = pd.read_csv("kl.csv", encoding='latin-1')

# Entendimiento del Dataset

Inicialmente se va a hacer un análisis de las columnas para entender que es lo que contiene cada columna, el tipo de dato que debe tener cada columna y, por supuesto, identificar aquellas que nos interesan

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
df.head()

,Unnamed: 0,ID,Name,Age,Photo,Nationality,Flag,Overall,Potential,Club,Club Logo,Value,Wage,Special,Preferred Foot,International Reputation,Weak Foot,Skill Moves,Work Rate,Body Type,Real Face,Position,Jersey Number,Joined,Loaned From,Contract Valid Until,Height,Weight,LS,ST,RS,LW,LF,CF,RF,RW,LAM,CAM,RAM,LM,LCM,CM,RCM,RM,LWB,LDM,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,Crossing,Finishing,HeadingAccuracy,ShortPassing,Volleys,Dribbling,Curve,FKAccuracy,LongPassing,BallControl,Acceleration,SprintSpeed,Agility,Reactions,Balance,ShotPower,Jumping,Stamina,Strength,LongShots,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Release Clause
0,0,158023,L. Messi,31.00,https://cdn.sofifa.org/players/4/19/158023.png,Argentina,https://cdn.sofifa.org/flags/52.png,94.00,94,FC Barcelona,https://cdn.sofifa.org/teams/2/light/241.png,110.5M,565K,2202,Left,5.00,4.00,4.00,Medium/ Medium,Messi,Yes,RF,10.00,"Jul 1, 2004",NaN,2021,5'7,159lbs,88+2,88+2,88+2,92+2,93+2,93+2,93+2,92+2,93+2,93+2,93+2,91+2,84+2,84+2,84+2,91+2,64+2,61+2,61+2,61+2,64+2,59+2,47+2,47+2,47+2,59+2,84.00,95.00,70.00,90.00,86.00,97.00,93.00,94.00,87.00,96.00,91.00,86.00,91.00,95.00,95.00,85.00,68.00,72.00,59.00,94.00,48.00,22.00,94.00,94.00,75.00,96.00,33.00,28.00,26.00,6.00,11.00,15.00,14.00,8.00,226.5M
1,1,20801,Cristiano Ronaldo,33.00,https://cdn.sofifa.org/players/4/19/20801.png,Portugal,https://cdn.sofifa.org/flags/38.png,94.00,94,Juventus,https://cdn.sofifa.org/teams/2/light/45.png,77M,405K,2228,Right,5.00,4.00,5.00,High/ Low,C. Ronaldo,Yes,ST,7.00,"Jul 10, 2018",NaN,2022,6'2,183lbs,91+3,91+3,91+3,89+3,90+3,90+3,90+3,89+3,88+3,88+3,88+3,88+3,81+3,81+3,81+3,88+3,65+3,61+3,61+3,61+3,65+3,61+3,53+3,53+3,53+3,61+3,84.00,94.00,89.00,81.00,87.00,88.00,81.00,76.00,77.00,94.00,89.00,91.00,87.00,96.00,70.00,95.00,95.00,88.00,79.00,93.00,63.00,29.00,95.00,82.00,85.00,95.00,28.00,31.00,23.00,7.00,11.00,15.00,14.00,11.00,127.1M
2,2,190871,Neymar Jr,26.00,https://cdn.sofifa.org/players/4/19/190871.png,Brazil,https://cdn.sofifa.org/flags/54.png,92.00,93,Paris Saint-Germain,https://cdn.sofifa.org/teams/2/light/73.png,118.5M,290K,2143,Right,5.00,5.00,5.00,High/ Medium,Neymar,Yes,LW,10.00,"Aug 3, 2017",NaN,2022,5'9,150lbs,84+3,84+3,84+3,89+3,89+3,89+3,89+3,89+3,89+3,89+3,89+3,88+3,81+3,81+3,81+3,88+3,65+3,60+3,60+3,60+3,65+3,60+3,47+3,47+3,47+3,60+3,79.00,87.00,62.00,84.00,84.00,96.00,88.00,87.00,78.00,95.00,94.00,90.00,96.00,94.00,84.00,80.00,61.00,81.00,49.00,82.00,56.00,36.00,89.00,87.00,81.00,94.00,27.00,24.00,33.00,9.00,9.00,15.00,15.00,11.00,228.1M
3,3,193080,De Gea,27.00,https://cdn.sofifa.org/players/4/19/193080.png,Spain,https://cdn.sofifa.org/flags/45.png,91.00,93,Manchester United,https://cdn.sofifa.org/teams/2/light/11.png,72M,260K,1471,Right,4.00,3.00,1.00,Medium/ Medium,Lean,Yes,GK,1.00,"Jul 1, 2011",NaN,2020,6'4,168lbs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17.00,13.00,21.00,50.00,13.00,18.00,21.00,19.00,51.00,42.00,57.00,58.00,60.00,90.00,43.00,31.00,67.00,43.00,64.00,12.00,38.00,30.00,12.00,68.00,40.00,68.00,15.00,21.00,13.00,90.00,85.00,87.00,88.00,94.00,138.6M
4,4,192985,K. De Bruyne,27.00,https://cdn.sofifa.org/players/4/19/192985.png,Belgium,https://cdn.sofifa.org/flags/7.png,91.00,92,Manchester City,https://cdn.sofifa.org/teams/2/light/10.png,102M,355K,2281,Right,4.00,5.00,4.00,High/ High,Normal,Yes,RCM,7.00,"Aug 30, 2015",NaN,2023,5'11,154lbs,82+3,82+3,82+3,87+3,87+3,87+3,87+3,87+3,88+3,88+3,88+3,88+3,87+3,87+3,87+3,88+3,77+3,77+3,77+3,77+3,77+3,73+3,66+3,66+3,66+3,73+3,93.00,82.00,55.00,92.00,82.00,86.00,85.00,83.00,91.00,91.00,78.00,76.00,79.00,91.00,77.00,91.00,63.00,90.00,75.00,91.00,76.00,61.00,87.00,94.00,79.00,88.00,68.00,58.00,51.00,15.00,13.00,5.00,10.00,13.00,196.4M


Tal como se puede ver hay algunas columnas que contienen datos no importantes para nuestro modelo, tal como:


*   'Unnamed: 0': La cual es un conteo que traia el csv internamente
*   'ID': El ID de cada jugador dentro del dataset
*   'Name': Nombre de cada jugador
*   'Photo': Foto del jugador
*   'Flag': Bandera del país de cada jugador
*   'Club Logo': Logo del club al que pertenece cada jugador
*   'Work Rate':
*   'Body Type': Tipo de cuerpo de cada jugador
*   'Real Face': Valor booleano que indica si el jugador tiene la cara real o un modelo 3D genérico
*   'Jersey Number': Número de camiseta del jugador en su equipo
*   'Joined': Fecha en que se unió a su club actual
*   'Loaned From': En case de que el jugador esté en préstamos, esta columna almacena el club que lo prestó






# Limpieza del Dataset

## Eliminar columnas que no nos interesan

In [ ]:
# Extraemos las columnas mencionadas anteriormente para eliminarlas del Dataset
non_important_columns = ['Unnamed: 0', 'ID', 'Name', 'Photo', 'Flag', 'Club Logo', 'Work Rate',
                         'Body Type', 'Real Face', 'Jersey Number', 'Joined', 'Loaned From']

In [ ]:
cleaned_df = df.drop(columns = non_important_columns)

## Decisiones sobre los datos con NaN

In [ ]:
def columns_with_nulls_cleaned():
  null_counts = cleaned_df.isnull().sum()
  columns_with_nulls = null_counts[null_counts > 0]

  print(columns_with_nulls)

In [ ]:
columns_with_nulls_cleaned()

Age                            1
Overall                        1
Club                         241
Preferred Foot                48
International Reputation      48
                            ... 
GKHandling                    48
GKKicking                     48
GKPositioning                 48
GKReflexes                    48
Release Clause              1564
Length: 72, dtype: int64


## Jugadores específicos

Se nota extraño que en el dataset haya un jugador con edad desconocida y uno con media desconocida.

Veamos quienes son

In [ ]:
df[df['Overall'].isnull()]

,Unnamed: 0,ID,Name,Age,Photo,Nationality,Flag,Overall,Potential,Club,Club Logo,Value,Wage,Special,Preferred Foot,International Reputation,Weak Foot,Skill Moves,Work Rate,Body Type,Real Face,Position,Jersey Number,Joined,Loaned From,Contract Valid Until,Height,Weight,LS,ST,RS,LW,LF,CF,RF,RW,LAM,CAM,RAM,LM,LCM,CM,RCM,RM,LWB,LDM,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,Crossing,Finishing,HeadingAccuracy,ShortPassing,Volleys,Dribbling,Curve,FKAccuracy,LongPassing,BallControl,Acceleration,SprintSpeed,Agility,Reactions,Balance,ShotPower,Jumping,Stamina,Strength,LongShots,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Release Clause
18,18,192448,M. ter Stegen,26.00,https://cdn.sofifa.org/players/4/19/192448.png,Germany,https://cdn.sofifa.org/flags/21.png,NaN,92,FC Barcelona,https://cdn.sofifa.org/teams/2/light/241.png,58M,240K,1328,Right,3.00,4.00,1.00,Medium/ Medium,Normal,Yes,GK,22.00,"Jul 1, 2014",NaN,2022,6'2,187lbs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.00,14.00,11.00,36.00,14.00,17.00,18.00,12.00,42.00,18.00,38.00,50.00,37.00,85.00,43.00,22.00,79.00,35.00,79.00,10.00,43.00,22.00,11.00,69.00,25.00,69.00,25.00,13.00,10.00,87.00,85.00,88.00,85.00,90.00,123.3M


In [ ]:
cleaned_df[cleaned_df['Overall'].isnull()]

,Age,Nationality,Overall,Potential,Club,Value,Wage,Special,Preferred Foot,International Reputation,Weak Foot,Skill Moves,Position,Contract Valid Until,Height,Weight,LS,ST,RS,LW,LF,CF,RF,RW,LAM,CAM,RAM,LM,LCM,CM,RCM,RM,LWB,LDM,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,Crossing,Finishing,HeadingAccuracy,ShortPassing,Volleys,Dribbling,Curve,FKAccuracy,LongPassing,BallControl,Acceleration,SprintSpeed,Agility,Reactions,Balance,ShotPower,Jumping,Stamina,Strength,LongShots,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Release Clause
18,26.00,Germany,NaN,92,FC Barcelona,58M,240K,1328,Right,3.00,4.00,1.00,GK,2022,6'2,187lbs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.00,14.00,11.00,36.00,14.00,17.00,18.00,12.00,42.00,18.00,38.00,50.00,37.00,85.00,43.00,22.00,79.00,35.00,79.00,10.00,43.00,22.00,11.00,69.00,25.00,69.00,25.00,13.00,10.00,87.00,85.00,88.00,85.00,90.00,123.3M


In [ ]:
df[df['Age'].isnull()]

,Unnamed: 0,ID,Name,Age,Photo,Nationality,Flag,Overall,Potential,Club,Club Logo,Value,Wage,Special,Preferred Foot,International Reputation,Weak Foot,Skill Moves,Work Rate,Body Type,Real Face,Position,Jersey Number,Joined,Loaned From,Contract Valid Until,Height,Weight,LS,ST,RS,LW,LF,CF,RF,RW,LAM,CAM,RAM,LM,LCM,CM,RCM,RM,LWB,LDM,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,Crossing,Finishing,HeadingAccuracy,ShortPassing,Volleys,Dribbling,Curve,FKAccuracy,LongPassing,BallControl,Acceleration,SprintSpeed,Agility,Reactions,Balance,ShotPower,Jumping,Stamina,Strength,LongShots,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Release Clause
11,11,182521,T. Kroos,NaN,https://cdn.sofifa.org/players/4/19/182521.png,Germany,https://cdn.sofifa.org/flags/21.png,90.00,90,Real Madrid,https://cdn.sofifa.org/teams/2/light/243.png,76.5M,355K,2190,Right,4.00,5.00,3.00,Medium/ Medium,Normal,Yes,LCM,8.00,"Jul 17, 2014",NaN,2022,6'0,168lbs,78+3,78+3,78+3,81+3,82+3,82+3,82+3,81+3,84+3,84+3,84+3,82+3,86+3,86+3,86+3,82+3,79+3,82+3,82+3,82+3,79+3,77+3,72+3,72+3,72+3,77+3,88.00,76.00,54.00,92.00,82.00,81.00,86.00,84.00,93.00,90.00,64.00,62.00,70.00,89.00,71.00,87.00,30.00,75.00,73.00,92.00,60.00,82.00,79.00,86.00,73.00,85.00,72.00,79.00,69.00,10.00,11.00,13.00,7.00,10.00,156.8M


In [ ]:
cleaned_df[cleaned_df['Age'].isnull()]

,Age,Nationality,Overall,Potential,Club,Value,Wage,Special,Preferred Foot,International Reputation,Weak Foot,Skill Moves,Position,Contract Valid Until,Height,Weight,LS,ST,RS,LW,LF,CF,RF,RW,LAM,CAM,RAM,LM,LCM,CM,RCM,RM,LWB,LDM,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,Crossing,Finishing,HeadingAccuracy,ShortPassing,Volleys,Dribbling,Curve,FKAccuracy,LongPassing,BallControl,Acceleration,SprintSpeed,Agility,Reactions,Balance,ShotPower,Jumping,Stamina,Strength,LongShots,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Release Clause
11,NaN,Germany,90.00,90,Real Madrid,76.5M,355K,2190,Right,4.00,5.00,3.00,LCM,2022,6'0,168lbs,78+3,78+3,78+3,81+3,82+3,82+3,82+3,81+3,84+3,84+3,84+3,82+3,86+3,86+3,86+3,82+3,79+3,82+3,82+3,82+3,79+3,77+3,72+3,72+3,72+3,77+3,88.00,76.00,54.00,92.00,82.00,81.00,86.00,84.00,93.00,90.00,64.00,62.00,70.00,89.00,71.00,87.00,30.00,75.00,73.00,92.00,60.00,82.00,79.00,86.00,73.00,85.00,72.00,79.00,69.00,10.00,11.00,13.00,7.00,10.00,156.8M


Como vemos que son Toni Kroos y Marc-André Ter Stegen, dos jugadores muy específicos podemos llenar esos campos con los datos obtenidos de Internet sobre ellos en FIFA 19

M. Ter Stegen: https://sofifa.com/player/192448/marc-andre-ter-stegen/190075

T. Kroos: https://sofifa.com/player/192448/marc-andre-ter-stegen/190075


In [ ]:
# Ter Ttegen Overall 89
cleaned_df.loc[18, 'Overall'] = 89

In [ ]:
# Toni Kroos Edad 28
cleaned_df.loc[11, 'Age'] = 28

In [ ]:
columns_with_nulls_cleaned()

Club                         241
Preferred Foot                48
International Reputation      48
Weak Foot                     48
Skill Moves                   48
                            ... 
GKHandling                    48
GKKicking                     48
GKPositioning                 48
GKReflexes                    48
Release Clause              1564
Length: 70, dtype: int64


## Jugadores sin equipo

Al analizar las columnas que tienen valores nulos se hace extraño ver que hay jugadores sin equipos. Veamos cuales son


Como se puede ver estos jugadores son datos atípicos que presentan 0 en todas las columnas relacionadas con dinero. Esto los convierte en datos sumamente atípicos por lo que lo mejor será eliminarlos para evitar posibles errores de medición a futuro.

In [ ]:
cleaned_df = cleaned_df[cleaned_df['Club'].notnull()]

## Jugadores con muchas estadísticas desconocidas

In [ ]:
columns_with_nulls_cleaned()

Preferred Foot                48
International Reputation      48
Weak Foot                     48
Skill Moves                   48
Position                      48
                            ... 
GKHandling                    48
GKKicking                     48
GKPositioning                 48
GKReflexes                    48
Release Clause              1323
Length: 69, dtype: int64


Ahora, notamos que hay exactamente 48 jugadores con NaN en muchas de las columnas. Veamos cuales son...

Como se ve, estos son jugadores de los que apenas se sabe algo, por lo que lo mejor será eliminarlos del dataset

In [ ]:
cleaned_df = cleaned_df[cleaned_df['Crossing'].notnull()]

## Decisión sobre los porteros

In [ ]:
columns_with_nulls_cleaned()

LS                1992
ST                1992
RS                1992
LW                1992
LF                1992
CF                1992
RF                1992
RW                1992
LAM               1992
CAM               1992
RAM               1992
LM                1992
LCM               1992
CM                1992
RCM               1992
RM                1992
LWB               1992
LDM               1992
CDM               1992
RDM               1992
RWB               1992
LB                1992
LCB               1992
CB                1992
RCB               1992
RB                1992
Release Clause    1275
dtype: int64


Podemos ver que quedan 1992 jugadores con datos en NaN. Sin embargo estos corresponden a aquellos que juegan de porteros. Veamos...

In [ ]:
cleaned_df[cleaned_df['LS'].isnull()].get('Position').describe()

count     1992
unique       1
top         GK
freq      1992
Name: Position, dtype: object

In [ ]:
cleaned_df = cleaned_df[cleaned_df['Position'] != 'GK']

Como eliminamos a los porteros ahora las columnas con estadísticas específicas para esta posición no son necesarias

In [ ]:
cleaned_df.columns

Index(['Age', 'Nationality', 'Overall', 'Potential', 'Club', 'Value', 'Wage',
       'Special', 'Preferred Foot', 'International Reputation', 'Weak Foot',
       'Skill Moves', 'Position', 'Contract Valid Until', 'Height', 'Weight',
       'LS', 'ST', 'RS', 'LW', 'LF', 'CF', 'RF', 'RW', 'LAM', 'CAM', 'RAM',
       'LM', 'LCM', 'CM', 'RCM', 'RM', 'LWB', 'LDM', 'CDM', 'RDM', 'RWB', 'LB',
       'LCB', 'CB', 'RCB', 'RB', 'Crossing', 'Finishing', 'HeadingAccuracy',
       'ShortPassing', 'Volleys', 'Dribbling', 'Curve', 'FKAccuracy',
       'LongPassing', 'BallControl', 'Acceleration', 'SprintSpeed', 'Agility',
       'Reactions', 'Balance', 'ShotPower', 'Jumping', 'Stamina', 'Strength',
       'LongShots', 'Aggression', 'Interceptions', 'Positioning', 'Vision',
       'Penalties', 'Composure', 'Marking', 'StandingTackle', 'SlidingTackle',
       'GKDiving', 'GKHandling', 'GKKicking', 'GKPositioning', 'GKReflexes',
       'Release Clause'],
      dtype='object')

In [ ]:
gk_columns = ['GKDiving', 'GKHandling', 'GKKicking', 'GKPositioning', 'GKReflexes']

In [ ]:
cleaned_df = cleaned_df.drop(columns = gk_columns)

## Jugadores con Release Clause en NaN

In [ ]:
columns_with_nulls_cleaned()

Release Clause    1183
dtype: int64


Como vemos, la única columna que aún tiene datos nulos es Release Clause. Esto sucede ya que se trata de jugadores que en su momento estaban de prestamo en sus clubes, veamos...

A estos jugadores vamos a eliminarlos...

In [ ]:
cleaned_df = cleaned_df[~cleaned_df['Release Clause'].isnull()]

In [ ]:
columns_with_nulls_cleaned()

Series([], dtype: int64)


## Conversión de datos numéricos

Una vez el data set ya no presenta datos con atributos nulos, debemos continuar con la conversión del tipo de los datos que se van a analizar a futuro.

### Conversión de valores monetarios

Como se puede ver, hay columnas que hacen referencia a valores monetarios. Es crucial convertir estas columnas que contienen valores en formato de texto a números, especialmente porque van a ser usados para nuestro modelo predictivo.

Las columnas en cuestión son: Value, Wage y Release Clause.

In [ ]:
def convert_currency(val):
  # Si val ya es numérico (int o float), simplemente devolverlo.
  if isinstance(val, (int, float)):
    return val

  # Asumir que si val no es numérico, debe ser una cadena.
  # Si no es una cadena o tiene menos de 2 caracteres, devolver 0.
  if not isinstance(val, str) or len(val) < 2:
    return 0

  val = val[1:]
  val_r = 0

  if 'M' in val:
    val_r = float(val.replace('M', ''))
    val_r = val_r * 1e6
  elif 'K' in val:
    val_r = float(val.replace('K', ''))
    val_r = val_r * 1e3
  else:
    # Si el valor no contiene ni 'M' ni 'K', intentamos convertirlo directamente a float.
    try:
      return float(val)
    except ValueError:
      return 0

  return val_r

In [ ]:
for col in ['Value', 'Wage', 'Release Clause']:
  cleaned_df[col] = cleaned_df[col].apply(convert_currency)

In [ ]:
cleaned_df[['Value', 'Wage', 'Release Clause']]

,Value,Wage,Release Clause
0,110500000.00,565000.00,226500000.00
1,77000000.00,405000.00,127100000.00
2,118500000.00,290000.00,228100000.00
4,102000000.00,355000.00,196400000.00
5,93000000.00,340000.00,172100000.00
...,...,...,...
18202,60000.00,1000.00,143000.00
18203,60000.00,1000.00,113000.00
18204,60000.00,1000.00,165000.00
18205,60000.00,1000.00,143000.00


### Conversión de calidad en diferentes posiciones

Las columnas 'LS', 'ST', 'RS', 'LW', 'LF', 'CF', 'RF', 'RW', 'LAM', 'CAM', 'RAM',
       'LM', 'LCM', 'CM', 'RCM', 'RM', 'LWB', 'LDM', 'CDM', 'RDM', 'RWB', 'LB',
       'LCB', 'CB', 'RCB', 'RB'; hacen referencia a la valoración de un jugador para jugar en esas posiciones, las cuales, en su mayoría, no son su posición natural.

Estas columnas presentan valores con formato X+Y por lo que remplazaremos ese valor con el resultado de dicha suma

In [ ]:
cleaned_df.columns

Index(['Age', 'Nationality', 'Overall', 'Potential', 'Club', 'Value', 'Wage',
       'Special', 'Preferred Foot', 'International Reputation', 'Weak Foot',
       'Skill Moves', 'Position', 'Contract Valid Until', 'Height', 'Weight',
       'LS', 'ST', 'RS', 'LW', 'LF', 'CF', 'RF', 'RW', 'LAM', 'CAM', 'RAM',
       'LM', 'LCM', 'CM', 'RCM', 'RM', 'LWB', 'LDM', 'CDM', 'RDM', 'RWB', 'LB',
       'LCB', 'CB', 'RCB', 'RB', 'Crossing', 'Finishing', 'HeadingAccuracy',
       'ShortPassing', 'Volleys', 'Dribbling', 'Curve', 'FKAccuracy',
       'LongPassing', 'BallControl', 'Acceleration', 'SprintSpeed', 'Agility',
       'Reactions', 'Balance', 'ShotPower', 'Jumping', 'Stamina', 'Strength',
       'LongShots', 'Aggression', 'Interceptions', 'Positioning', 'Vision',
       'Penalties', 'Composure', 'Marking', 'StandingTackle', 'SlidingTackle',
       'Release Clause'],
      dtype='object')

In [ ]:
def sum_position(pos):

  num = pos.split("+")

  if len(num) < 2:
    try:
      return num[0]
    except:
      return 0

  res = float(num[0]) + float(num[1])

  return res


In [ ]:
for col in ['LS', 'ST', 'RS', 'LW', 'LF', 'CF', 'RF', 'RW', 'LAM', 'CAM', 'RAM',
       'LM', 'LCM', 'CM', 'RCM', 'RM', 'LWB', 'LDM', 'CDM', 'RDM', 'RWB', 'LB',
       'LCB', 'CB', 'RCB', 'RB']:
  cleaned_df[col] = cleaned_df[col].apply(sum_position)

### Conversión de Height y Weight

Las columnas de Heigth y Weight tienen la falla de presentar unidades, por lo que lo mejor será mostrar solo su representación numérica.

In [ ]:
#Convertir Height a cm
def height_to_cm(height_str):
    # Dividir la cadena en pies y pulgadas
    feet, inches = map(int, height_str.split("'"))

    # Convertir a centímetros
    total_inches = (feet * 12) + inches
    cm = total_inches * 2.54  # 1 pulgada es aproximadamente 2.54 cm

    return cm

In [ ]:
cleaned_df['Height'] = cleaned_df['Height'].apply(height_to_cm)

In [ ]:
#Convertir Weight a kg
def weight_to_kg(weight_str):
    # Eliminar el sufijo 'lbs' y convertir a float
    pounds = float(weight_str.replace('lbs', ''))

    # Convertir a kilogramos
    kg = pounds * 0.453592  # 1 libra es aproximadamente 0.453592 kilogramos

    return kg

In [ ]:
cleaned_df['Weight'] = cleaned_df['Weight'].apply(weight_to_kg)

## Conversión de columnas categóricas

Finalmente, el dataset presenta 4 columnas categóricas: Nationality, Club, Preferred Foot y Position

Empecemos por el más sencillo, Preferred Foot.

Esta columna solo presenta dos respuestas: Left y Right, por lo que remplazaremos Left por 0 y Right por 1

In [ ]:
cleaned_df['Preferred Foot'] = cleaned_df['Preferred Foot'].replace({'Right': 1, 'Left': 0})

In [ ]:
pd.set_option('display.float_format', '{:.2f}'.format)

Ahora para Nationality y Club considero que lo mejor es remplazarlo por la media del valor de sus jugadores

In [ ]:
#Media para Club
club_value_mean = cleaned_df.groupby('Club')['Value'].mean()

In [ ]:
cleaned_df['Mean Club Value'] = cleaned_df['Club'].map(club_value_mean)

In [ ]:
#Eliminamos la columna Club que es categórica
cleaned_df.drop('Club', axis=1, inplace=True)

In [ ]:
#Media para Nationality
nationality_value_mean = cleaned_df.groupby('Nationality')['Value'].mean()

cleaned_df['Mean Nationality Value'] = cleaned_df['Nationality'].map(nationality_value_mean)

#Eliminamos la columna Nationality que es categórica
cleaned_df.drop('Nationality', axis=1, inplace=True)

Finalmente, para Position decidimos quitarla ya que con la calidad específica para cada posición es suficiente

In [ ]:
cleaned_df = cleaned_df.drop(columns = ["Position"])

In [ ]:
cleaned_df

,Age,Overall,Potential,Value,Wage,Special,Preferred Foot,International Reputation,Weak Foot,Skill Moves,Contract Valid Until,Height,Weight,LS,ST,RS,LW,LF,CF,RF,RW,LAM,CAM,RAM,LM,LCM,CM,RCM,RM,LWB,LDM,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,Crossing,Finishing,HeadingAccuracy,ShortPassing,Volleys,Dribbling,Curve,FKAccuracy,LongPassing,BallControl,Acceleration,SprintSpeed,Agility,Reactions,Balance,ShotPower,Jumping,Stamina,Strength,LongShots,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Marking,StandingTackle,SlidingTackle,Release Clause,Mean Club Value,Mean Nationality Value
0,31.00,94.00,94,110500000.00,565000.00,2202,0,5.00,4.00,4.00,2021,170.18,72.12,90.00,90.00,90.00,94.00,95.00,95.00,95.00,94.00,95.00,95.00,95.00,93.00,86.00,86.00,86.00,93.00,66.00,63.00,63.00,63.00,66.00,61.00,49.00,49.00,49.00,61.00,84.00,95.00,70.00,90.00,86.00,97.00,93.00,94.00,87.00,96.00,91.00,86.00,91.00,95.00,95.00,85.00,68.00,72.00,59.00,94.00,48.00,22.00,94.00,94.00,75.00,96.00,33.00,28.00,26.00,226500000.00,26745689.66,3282685.56
1,33.00,94.00,94,77000000.00,405000.00,2228,1,5.00,4.00,5.00,2022,187.96,83.01,94.00,94.00,94.00,92.00,93.00,93.00,93.00,92.00,91.00,91.00,91.00,91.00,84.00,84.00,84.00,91.00,68.00,64.00,64.00,64.00,68.00,64.00,56.00,56.00,56.00,64.00,84.00,94.00,89.00,81.00,87.00,88.00,81.00,76.00,77.00,94.00,89.00,91.00,87.00,96.00,70.00,95.00,95.00,88.00,79.00,93.00,63.00,29.00,95.00,82.00,85.00,95.00,28.00,31.00,23.00,127100000.00,29126136.36,5312205.32
2,26.00,92.00,93,118500000.00,290000.00,2143,1,5.00,5.00,5.00,2022,175.26,68.04,87.00,87.00,87.00,92.00,92.00,92.00,92.00,92.00,92.00,92.00,92.00,91.00,84.00,84.00,84.00,91.00,68.00,63.00,63.00,63.00,68.00,63.00,50.00,50.00,50.00,63.00,79.00,87.00,62.00,84.00,84.00,96.00,88.00,87.00,78.00,95.00,94.00,90.00,96.00,94.00,84.00,80.00,61.00,81.00,49.00,82.00,56.00,36.00,89.00,87.00,81.00,94.00,27.00,24.00,33.00,228100000.00,22380555.56,4660353.68
4,27.00,91.00,92,102000000.00,355000.00,2281,1,4.00,5.00,4.00,2023,180.34,69.85,85.00,85.00,85.00,90.00,90.00,90.00,90.00,90.00,91.00,91.00,91.00,91.00,90.00,90.00,90.00,91.00,80.00,80.00,80.00,80.00,80.00,76.00,69.00,69.00,69.00,76.00,93.00,82.00,55.00,92.00,82.00,86.00,85.00,83.00,91.00,91.00,78.00,76.00,79.00,91.00,77.00,91.00,63.00,90.00,75.00,91.00,76.00,61.00,87.00,94.00,79.00,88.00,68.00,58.00,51.00,196400000.00,25589137.93,5253652.85
5,27.00,91.00,91,93000000.00,340000.00,2142,1,4.00,4.00,4.00,2020,172.72,73.94,86.00,86.00,86.00,92.00,91.00,91.00,91.00,92.00,92.00,92.00,92.00,92.00,85.00,85.00,85.00,92.00,69.00,66.00,66.00,66.00,69.00,63.00,52.00,52.00,52.00,63.00,81.00,84.00,61.00,89.00,80.00,95.00,83.00,79.00,83.00,94.00,94.00,88.00,95.00,90.00,94.00,82.00,56.00,83.00,66.00,80.00,54.00,41.00,87.00,89.00,86.00,91.00,34.00,27.00,22.00,172100000.00,19355535.71,5253652.85
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18202,19.00,47.00,65,60000.00,1000.00,1307,1,1.00,2.00,2.00,2019,175.26,60.78,44.00,44.00,44.00,46.00,46.00,46.00,46.00,46.00,47.00,47.00,47.00,46.00,47.00,47.00,47.00,46.00,46.00,47.00,47.00,47.00,46.00,47.00,47.00,47.00,47.00,47.00,34.00,38.00,40.00,49.00,25.00,42.00,30.00,34.00,45.00,43.00,54.00,57.00,60.00,49.00,76.00,43.00,55.00,40.00,47.00,38.00,46.00,46.00,39.00,52.00,43.00,45.00,40.00,48.00,47.00,143000.00,190208.33,1621613.64
18203,19.00,47.00,63,60000.00,1000.00,1098,1,1.00,2.00,2.00,2020,190.50,77.11,47.00,47.00,47.00,41.00,44.00,44.00,44.00,41.00,42.00,42.00,42.00,40.00,37.00,37.00,37.00,40.00,32.00,33.00,33.00,33.00,32.00,31.00,34.00,34.00,34.00,31.00,23.00,52.00,52.00,43.00,36.00,39.00,32.00,20.00,25.00,40.00,41.00,39.00,38.00,40.00,52.00,41.00,47.00,43.00,67.00,42.00,47.00,16.00,46.00,33.00,43.00,42.00,22.00,15.00,19.00,113000.00,267708.33,1174365.78
18204,16.00,47.00,6

# Guardar el dataset procesado

El propósito de esto notebook era entender el dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

cleaned_df.to_csv("/content/drive/My Drive/fifa-19-cleaned-dataset.csv", index=False)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
